In [2]:
from pyphy import pyphy
import re
import pandas as pd

In [3]:
rx_entry = re.compile(r'(\d+)\s+(.+)\[TAX:(\d+)\]')

In [4]:
virus_strand = {}

with open("brite/brite_virus.txt", "r") as input:
    strand = ""
    for line in input:
        #print (line.strip())

        if line.startswith("A"):
            strand = line.replace("A", "").strip()
        elif line[0] in ["#", "!", "+"]:
            pass
        else:
            search_entry = rx_entry.search(line)
            if not search_entry:
                line = line[1:].strip()
                
                if line not in virus_strand:
                    virus_strand[line] = [strand]
                else:
                    virus_strand[line] += [strand]

In [5]:
taxon = pd.read_csv("brite/import/taxon.csv")

In [6]:
desired_rank = ["superkingdom", "clade", "phylum", "class", "order", "family", "genus", "species", "genome"][::-1]

desired_rank
    

['genome',
 'species',
 'genus',
 'family',
 'order',
 'class',
 'phylum',
 'clade',
 'superkingdom']

In [7]:
content = "type,taxid\n"
for  index, line in taxon.iterrows():
    taxid = line["taxid"]

    dict_path = pyphy.getDictPathByTaxid(taxid)

    for rank in desired_rank:

        if rank in dict_path:
            taxon_name = pyphy.getNameByTaxid(dict_path[rank])
            #print (taxon_name)
            if taxon_name in virus_strand:
                strand = virus_strand[taxon_name]
                for s in strand:
                    content += f'{s},{taxid}\n'
                #print (f'{taxid},{line["name"]},{line["rank"]},{strand}\n')
                break



In [9]:
with open("brite/import/type_taxon_connections.csv", "w") as output:
    output.write(content)

In [10]:
type_df = pd.read_csv("brite/import/type_taxon_connections.csv")

In [19]:
list(type_df.type.unique())

['dsDN-RT viruses',
 'ssRN-RT viruses',
 'dsRN viruses',
 '+ssRN viruses',
 '-ssRN viruses',
 'Other viruses',
 'dsDN viruses',
 'ssDN viruses']

In [20]:
with open("brite/import/type.csv", "w") as output:
    output.write("name\n" + "\n".join(list(type_df.type.unique())))